# FRED Economic Data Scraping Description

The script consists of a class `FREDAPIFetcher` to interact with the FRED API and fetch data for specified series. The main function, `fetch_gross_value_added_data`, retrieves GVA data for various sectors, processes the data, and saves it to a CSV file.

## FREDAPIFetcher Class

- **`__init__(self, api_key)`**: Initializes the class with the provided API key.
- **`fetch_series_data(self, series_id, start_date, end_date)`**: Fetches data for the specified series ID between the given start and end dates, processes the data into a pandas DataFrame, and returns it.

## fetch_gross_value_added_data Function

- **`fetch_gross_value_added_data(api_key)`**: Creates an instance of `FREDAPIFetcher`, defines the series IDs for various sectors, and fetches the data for each sector. The data is then combined into a single DataFrame and returned.

## Main Execution Block

- Replaces `'YOUR_API_KEY'` with your actual FRED API key.
- Calls `fetch_gross_value_added_data` and saves the resulting DataFrame to a CSV file.

## Instructions

### Set Up Your Environment

Ensure you have Python 3.x installed and the required libraries (`pandas` and `requests`) installed.

### Get Your FRED API Key

Sign up at [FRED](https://fred.stlouisfed.org/) to get an API key.

### Update the Script

Replace `'YOUR_API_KEY'` in the script with your actual FRED API key.


In [2]:
import pandas as pd
import requests
from datetime import datetime
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class FREDAPIFetcher:
    def __init__(self, api_key):
        self.api_key = api_key
        self.base_url = 'https://api.stlouisfed.org/fred/'

    def fetch_series_data(self, series_id, start_date, end_date):
        endpoint = 'series/observations'
        params = {
            'series_id': series_id,
            'api_key': self.api_key,
            'file_type': 'json',
            'observation_start': start_date,
            'observation_end': end_date,
            'frequency': 'q',  # Quarterly data
            'units': 'lin',  # Linear units (not percent change)
        }

        response = requests.get(self.base_url + endpoint, params=params)

        if response.status_code == 200:
            data = response.json()
            df = pd.DataFrame(data['observations'])
            df['date'] = pd.to_datetime(df['date'])
            df['value'] = pd.to_numeric(df['value'], errors='coerce')
            df.set_index('date', inplace=True)
            return df
        else:
            logger.error(f'Failed to retrieve data for {series_id}. Status code: {response.status_code}')
            return None

def fetch_gross_value_added_data(api_key):
    fetcher = FREDAPIFetcher(api_key)
    
    # Updated series IDs for gross value added by sector
    series_ids = {
        'Gross domestic product': 'GDP',
        'Business': 'A195RC1Q027SBEA',
        'Nonfarm': 'A358RC1Q027SBEA',
        'Farm': 'B359RC1Q027SBEA',
        'Households and institutions': 'A193RC1Q027SBEA',
        'Nonprofit institutions serving households': 'B702RC1Q027SBEA',
        'General government': 'A765RC1Q027SBEA',
        'Federal': 'A766RC1Q027SBEA',
        'State and local': 'A767RC1Q027SBEA',
        'Gross housing value added': 'A2009C1Q027SBEA'
    }

    start_date = '1950-01-01'
    end_date = '2024-01-01'

    data = {}
    for sector, series_id in series_ids.items():
        sector_data = fetcher.fetch_series_data(series_id, start_date, end_date)
        if sector_data is not None and not sector_data.empty:
            data[sector] = sector_data['value']
        else:
            logger.warning(f'No data available for {sector} (Series ID: {series_id})')

    df = pd.DataFrame(data)
    
    return df

if __name__ == "__main__":
    # Replace 'YOUR_API_KEY' with your actual FRED API key
    api_key = '3f2128626a075c9881181a2a49811b1d'
    
    try:
        data = fetch_gross_value_added_data(api_key)
        if data is not None:
            print(data)
            
            # Save to CSV
            data.to_csv(r'D:\Downloads\gva_data_1950_2024.csv')
            logger.info("Data saved to D:\\Downloads\\gva_data_1950_2024.csv")
        else:
            logger.error("No data fetched.")
    except Exception as e:
        logger.error(f"An error occurred: {str(e)}")



INFO:__main__:Data saved to D:\Downloads\gva_data_1950_2024.csv


            Gross domestic product   Business    Nonfarm     Farm  \
date                                                                
1950-01-01                 280.828    232.582    214.879   17.703   
1950-04-01                 290.383    241.558    223.656   17.902   
1950-07-01                 308.153    257.435    238.324   19.111   
1950-10-01                 319.945    266.085    245.899   20.186   
1951-01-01                 336.000    278.530    257.330   21.200   
...                            ...        ...        ...      ...   
2023-01-01               26813.601  20657.951  20435.618  222.333   
2023-04-01               27063.012  20813.738  20606.096  207.642   
2023-07-01               27610.128  21248.644  21049.305  199.339   
2023-10-01               27956.998  21491.702  21307.177  184.525   
2024-01-01               28269.174  21692.313  21509.776  182.537   

            Households and institutions  \
date                                      
1950-01-01      